## Chroma Vector DB の使い方
#### 前提条件: create_chroma_job.ipynb から、populate_chroma_vectors.py の実行が終わっていること。
講習では、このプロセスは講師が事前に実行しています。

#### 5.1 Chroma Vector DB のコネクションを初期化

In [ ]:
## Chroma DB サーバーとの接続を初期化
import chromadb
import os

## CMLの中から呼び出すために、以下のコードを利用
chroma_client = chromadb.PersistentClient(path="/home/cdsw/chroma-data")

#### 5.2 Chroma DB の ベクターコレクションと、オブジェクトの数を取得
以下のコードでは、Chroma DBへの接続を初期化します。  
使用するエンべディングモデルを定義し、コレクション名を'cml-default'と指定し、指定された埋め込み関数でそのコレクションの取得や作成を行います。  
最後に、Chroma DBのインデックスに含まれるエンベディングの総数を取得・表示し、コレクションの統計情報を表示します。

In [ ]:
from chromadb.utils import embedding_functions

EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"
EMBEDDING_MODEL_NAME = "all-mpnet-base-v2"
EMBEDDING_FUNCTION = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBEDDING_MODEL_NAME)

COLLECTION_NAME = 'cml-default'

print("Chroma DB との接続を初期化しています ...")

print(f"'{COLLECTION_NAME}' をオブジェクトとして取得しています...")
try:
    chroma_client.get_collection(name=COLLECTION_NAME, embedding_function=EMBEDDING_FUNCTION)
    print("Success")
    collection = chroma_client.get_collection(name=COLLECTION_NAME, embedding_function=EMBEDDING_FUNCTION)
except:
    print("新規の接続を作成しています...")
    collection = chroma_client.create_collection(name=COLLECTION_NAME, embedding_function=EMBEDDING_FUNCTION)
    print("Success")

# 最新の統計情報を表示
current_collection_stats = collection.count()
print('Chroma DB の index 内のエンべディング数 : ' + str(current_collection_stats))

#### 5.3 複数の属性を指定してChromaにベクトルを入力するサンプルデモ

ここでは、指定されたテキスト、分類(classification)、ファイルパスを使用して、関連するメタデータと一意のIDを持つサンプルドキュメントを、ベクトル検索のためにChromaベクトルデータベースコレクションに追加します。

In [ ]:
## !!! このコードは、ハンズオンの中では実行しないでください !!! ###

## Chroma vector DB にデータを追加する例 
file_path = '/example/of/file/path/to/doc.txt'
classification = "public"
text = "これはベクトルDBに格納する文章のサンプルです。"

collection.add(
    documents=[text],
    metadatas=[{"classification": classification}],
    ids=[file_path]
)

In [ ]:
## 上記のセルを誤って実行してしまった場合は、こちらのセルを実行してください。

# 削除したいIDを指定
id_to_delete = '/example/of/file/path/to/doc.txt'

# IDに関連付けられたレコードを削除
delete_response = collection.delete(ids=[id_to_delete])

# 削除結果を確認
print(delete_response)

#### 5.4 Chromaでベクトルをクエリし、メタデータを使ってノイズを減らすサンプルデモ

このコードは、サンプルのクエリテキストを使用してChromaベクトルデータベースのセマンティック検索（＝ベクトルが近いものを取得することで、結果として意味が近いものを取得する）を実行し、最も類似した2つの結果を取得します。  
メタデータによるフィルタを指定することにより、検索結果をさらに絞り込みでき、より正確でコンテキストを考慮したクエリを可能にします。

In [ ]:
## Chroma vector DB をクエリする
## このクエリは、最も意味が近いふたつのベクトルを取得します。
results = collection.query(
    query_texts=["Apache Iceberg って何ですか?"],
    n_results=2
    # where={"metadata_field": "is_equal_to_this"}, # メタデータによる絞り込みの例（任意）
    # where_document={"$contains":"search_string"}  # 〃
)
print(results)

#### 5.5 Chromaを使用してローカルファイルシステム内のオリジナルファイル（完全なファイル）にマッピングした結果

このコードでは、ヘルパー関数load_context_chunk_from_dataを定義し、ファイルパス（ID）に基づいて知識ベースドキュメントのコンテンツを取得しています。
検索結果を繰り返し実行して、ファイルパス、分類、ドキュメントのスニペット、ファイルからロードされた完全なドキュメントコンテンツなど、各結果に関する情報を表示し、検索結果の詳細な表示を提供します。

In [ ]:
# ナレッジベースをID（ファイルパス）に基づいて取得するためのヘルパー関数
def load_context_chunk_from_data(id_path):
    with open(id_path, "r") as f: # 読み込みモードでファイルを開く
        return f.read()
    
## ファイルの表示
for i in range(len(results['ids'][0])):
    file_path = results['ids'][0][i]
    classification = results['metadatas'][0][i]['classification']
    document = results['documents'][0][i]
    
    print("------------- 結果 " + str(i+1) + " ----------------\n")
    print(f"ファイルパス: {file_path}")
    print(f"分類: {classification}")
    print(f"ドキュメント: {document}\n")
    print(f"ドキュメント全文 (ファイルより取得): {load_context_chunk_from_data(file_path)}\n")
